<a href="https://colab.research.google.com/github/akancijan/Pet-Detection-Model/blob/main/TF2model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import glob
import xml.etree.ElementTree as ET
import pandas as pd
import tensorflow as tf

from shutil import copyfile
print(tf.__version__)

2.7.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
os.chdir('/content')
print(os.listdir())

In [ ]:
os.mkdir('models') 
print(os.listdir())

In [ ]:
# Split pictures and XML files in seperate folders

# src_directory = '/content/drive/MyDrive/Diplomski/Asirra_ cat vs dogs'
# for file in os.listdir(src_directory):
#     if '.jpg' in str(file):
#       src_jpg = src_directory + '/' + file
#       xml_file = str(file).replace('.jpg','.xml')
#       src_xml = src_directory + '/' + xml_file
#       dst_dir_img = '/content/drive/MyDrive/Diplomski/customTF2/images/'
#       dst_dir_xml = '/content/drive/MyDrive/Diplomski/customTF2/annotations/'
#       print(f'src_jpg {src_jpg} ')
#       print(f'src_xml {src_xml} ')
#       print(f'dst_img {dst_dir_img}')
#       print(f'dst_xml {dst_dir_xml}')
#       if file.startswith('cat'):
#           dst_jpg = dst_dir_img + file
#           dst_xml = dst_dir_xml + xml_file
#           print(f'Send cat picture to : {dst_jpg}')
#           print(f'Send cat xml to : {dst_xml}')
#           copyfile(src_jpg, dst_jpg)
#           copyfile(src_xml, dst_xml)
#       elif file.startswith('dog'):
#           dst_jpg =dst_dir_img + file
#           dst_xml =dst_dir_xml + xml_file
#           print(f'Send dog picture to : {dst_jpg}')
#           print(f'Send dog xml to : {dst_xml}')
#           copyfile(src_jpg, dst_jpg)
#           copyfile(src_xml, dst_xml)

In [ ]:
# Symbolic link = /content/drive/MyDrive/Diplomski/ is now /mydrive
!ln -s /content/drive/MyDrive/Diplomski/ /mydrive
!ls /mydrive

In [ ]:
# clone the tensorflow models on the colab cloud vm
!git clone --q https://github.com/tensorflow/models.git

#navigate to /models/research folder to compile protos
%cd models/research

# Compile protos.
!protoc object_detection/protos/*.proto --python_out=.

# Install TensorFlow Object Detection API.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .


In [ ]:
# testing the model builder
!python object_detection/builders/model_builder_tf2_test.py

In [ ]:
# Setup traning and validation sets
#
# %cd /mydrive/customTF2/data/

# !mkdir test_labels train_labels

In [ ]:
# #   Note!!!
# ### run this once to sort pictures
# ### after that you can skip this section 
# import random
# from random import seed
# from random import randint

# # seed random number generator
# seed(1)
# # Set aside 20% of labeled pictures for validation of our created model
# val_ratio = 0.20
# # Sort pictures into test and train directories
# # and further sort them by dog/cat subdirectories
# src_directory = '/content/drive/MyDrive/Diplomski/customTF2/annotations/'
# for file in os.listdir(src_directory):
#     if '.xml' in str(file):
#       src_xml = src_directory + file
#       dst_dir = '/content/drive/MyDrive/Diplomski/customTF2/data/train_labels/'
#       print(f'src xml {src_xml}')
#       print(f'dst {dst_dir}')
#       if random.random() < val_ratio:
#           dst_dir = '/content/drive/MyDrive/Diplomski/customTF2/data/test_labels/'
#           print(f'random triggered : {dst_dir}')
#       if file.startswith('cat'):
#           dst_xml = dst_dir + file
#           print(f'Send cat xml to : {dst_xml}')
#           copyfile(src_xml, dst_xml)

#       elif file.startswith('dog'):
#           dst_xml = dst_dir + file
#           print(f'Send dog xml to : {dst_xml}')
#           copyfile(src_xml, dst_xml)

In [ ]:
# Check if training set has about the same amount of cat/dog labels

src_directory = '/content/drive/MyDrive/Diplomski/customTF2/data/train_labels/'
dog_count=0
cat_count=0
for file in os.listdir(src_directory):
  if 'cat' in str(file):
    cat_count+=1
  if 'dog' in str(file):
    dog_count+=1
  

print(f'Dogs : {dog_count} Cats {cat_count}')

# Check if test set has about the same amount of cat/dog labels

src_directory = '/content/drive/MyDrive/Diplomski/customTF2/data/test_labels/'
dog_count=0
cat_count=0
for file in os.listdir(src_directory):
  if 'cat' in str(file):
    cat_count+=1
  if 'dog' in str(file):
    dog_count+=1
  

print(f'Dogs : {dog_count} Cats {cat_count}')

In [ ]:
# Read in .xml files to a combined .csv file for record creation
def xml_to_csv(path):
  classes_names = []
  xml_list = []

  for xml_file in glob.glob(path + '/*.xml'):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    for member in root.findall('object'):
      classes_names.append(member[0].text)
      value = (root.find('filename').text  ,   
               int(root.find('size')[0].text),
               int(root.find('size')[1].text),
               member[0].text,
               float(member[4][0].text),
               float(member[4][1].text),
               float(member[4][2].text),
               float(member[4][3].text))
      xml_list.append(value)
  column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
  xml_df = pd.DataFrame(xml_list, columns=column_name) 
  classes_names = list(set(classes_names))
  classes_names.sort()
  return xml_df, classes_names

for label_path in ['train_labels', 'test_labels']:
  image_path = os.path.join(os.getcwd(), label_path)
  xml_df, classes = xml_to_csv(label_path)
  xml_df.to_csv(f'{label_path}.csv', index=None)
  print(f'Successfully converted {label_path} xml to csv.')

label_map_path = os.path.join("label_map.pbtxt")
pbtxt_content = ""

for i, class_name in enumerate(classes):
    pbtxt_content = (
        pbtxt_content
        + "item {{\n    id: {0}\n    name: '{1}'\n}}\n\n".format(i + 1, class_name)
    )
pbtxt_content = pbtxt_content.strip()
with open(label_map_path, "w") as f:
    f.write(pbtxt_content)
    print('Successfully created label_map.pbtxt ')   

In [ ]:
%cd /mydrive/customTF2/data

In [ ]:
# Run downloaded script for creation of .record files
# Use it twice to create train and test records

#For train.record
!python /mydrive/customTF2/generate_tfrecord.py train_labels.csv  label_map.pbtxt /mydrive/customTF2/images/ train.record

#For test.record
!python /mydrive/customTF2/generate_tfrecord.py test_labels.csv  label_map.pbtxt /mydrive/customTF2/images/ test.record


groups: 100% 898/898 [00:02<00:00, 425.52it/s]
Successfully created the TFRecords: /content/drive/MyDrive/Diplomski/customTF2/data/train.record
groups: 100% 202/202 [00:00<00:00, 441.54it/s]
Successfully created the TFRecords: /content/drive/MyDrive/Diplomski/customTF2/data/test.record


In [ ]:
# Download the pre-trained model 
# For this dataset we are using faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz 
# Download it into the data folder & unzip it.

!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz
!tar -xzvf faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz

--2022-01-22 19:04:59--  http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.251.6.128, 2607:f8b0:4001:c5a::80
Connecting to download.tensorflow.org (download.tensorflow.org)|142.251.6.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 211996178 (202M) [application/x-tar]
Saving to: ‘faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz’

faster_rcnn_resnet5 100%[===================>] 202.17M  64.2MB/s    in 3.2s    

2022-01-22 19:05:02 (64.2 MB/s) - ‘faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz’ saved [211996178/211996178]

faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/
faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/checkpoint/
faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/checkpoint/checkpoint
faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/chec

In [ ]:
#load tensorboard

%load_ext tensorboard         
%tensorboard --logdir '/content/drive/MyDrive/Diplomski/customTF2/training'

In [ ]:
%cd /content/models/research/object_detection

/content/models/research/object_detection


In [ ]:
# Need to downgrade opencv-python-headless package 
# Higher versions cause an error that prevents the model from training
!pip install "opencv-python-headless<4.3"

     |████████████████████████████████| 21.6 MB 6.2 MB/s 
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.5.5.62
    Uninstalling opencv-python-headless-4.5.5.62:
      Successfully uninstalled opencv-python-headless-4.5.5.62


In [ ]:
# Run the command below from the content/models/research/object_detection directory

# PIPELINE_CONFIG_PATH={path to pipeline config file}
# MODEL_DIR={path to model directory}
# Example of the python command to run the training process
#!python model_main_tf2.py --pipeline_config_path=${PIPELINE_CONFIG_PATH} --model_dir=${MODEL_DIR} --alsologtostderr

!python model_main_tf2.py --pipeline_config_path=/mydrive/customTF2/data/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.config --model_dir=/mydrive/customTF2/training --alsologtostderr

In [ ]:
## Export inference graph
!python exporter_main_v2.py --trained_checkpoint_dir=/mydrive/customTF2/training --pipeline_config_path=/mydrive/customTF2/data/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.config --output_directory /mydrive/customTF2/data/inference_graph

In [ ]:
# Different font-type for labels text
!wget https://freefontsdownload.net/download/160187/arial.zip
!unzip arial.zip -d .

%cd utils/
!sed -i "s/font = ImageFont.truetype('arial.ttf', 24)/font = ImageFont.truetype('arial.ttf', 50)/" visualization_utils.py
%cd ..

In [ ]:
#Loading the saved_model
import tensorflow as tf
import time
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from PIL import Image
from google.colab.patches import cv2_imshow
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

IMAGE_SIZE = (16, 8) # Output display size as you want
import matplotlib.pyplot as plt
PATH_TO_SAVED_MODEL="/mydrive/customTF2/data/inference_graph/saved_model"
print('Loading model...', end='')

# Load saved model and build the detection function
detect_fn=tf.saved_model.load(PATH_TO_SAVED_MODEL)
print('Done!')

# Loading the label_map
category_index=label_map_util.create_category_index_from_labelmap("/mydrive/customTF2/data/label_map.pbtxt",use_display_name=True)

def load_image_into_numpy_array(path):

    return np.array(Image.open(path))


Loading model...Done!


In [ ]:
from operator import itemgetter
# Testing out model on single images

image_path = "/mydrive/customTF2/uploaded_pictures/moki_sunset.jpg"

image_np = load_image_into_numpy_array(image_path)

# The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
input_tensor = tf.convert_to_tensor(image_np)
# The model expects a batch of images, so add an axis with `tf.newaxis`.
input_tensor = input_tensor[tf.newaxis, ...]

detections = detect_fn(input_tensor)





# All outputs are batches tensors.
# Convert to numpy arrays, and take index [0] to remove the batch dimension.
# We're only interested in the first num_detections.
num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_detections,
      detections['detection_boxes'],
      detections['detection_classes'],
      detections['detection_scores'],
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=200,
      min_score_thresh=.4, # Adjust this value to set the minimum probability boxes to be classified as True
      agnostic_mode=False)


#Create indexes list of element with a score > 0.5
indexes = [k for k,v in enumerate(detections['detection_scores']) if (v > 0.2)]

#Number of entities
num_entities = len(indexes)
#Extract the class id
class_id = itemgetter(*indexes)(detections['detection_classes'])
scores = itemgetter(*indexes)(detections['detection_scores'])

#Convert the class id in their name
class_names = []
if num_entities == 1:
  class_names.append(category_index[class_id]['name'])
  class_name = str(class_names)
else:
  for i in range(0, len(indexes)):
    class_names.append(category_index[class_id[i]]['name'])

print(class_names)
print(class_id)
print(scores)




%matplotlib inline
plt.figure(figsize=IMAGE_SIZE, dpi=200)
plt.axis("off")
plt.imshow(image_np_with_detections)
plt.show()



In [ ]:
images_dir = "/mydrive/customTF2/uploaded_pictures/"
# simple version for working with CWD
number_of_pictures = 0
for file in os.listdir(images_dir):
  number_of_pictures += 1
print(number_of_pictures)

23


In [ ]:
!pip install xlsxwriter
import xlsxwriter
import re

report_path = "/mydrive/customTF2/excel_report"

workbook = xlsxwriter.Workbook(f'{report_path}/guesses4.xlsx')
worksheet = workbook.add_worksheet(f'Dog-Cat Guesses')

# Column names
worksheet.write('A1', 'Picture_ID')
worksheet.write('B1', 'Class')
worksheet.write('C1', 'Score')
worksheet.write('D1', 'Picture_Nr')

row_index = 2

In [ ]:
# Rows are equal to "number_of_pictures" to form the wanted subplot
rows = number_of_pictures
columns = 1
file_number = 1

images_dir_testing = "/mydrive/customTF2/cat-dog_pictures_with_defined_name/"

for file in os.listdir(images_dir_testing):
  image_path = images_dir_testing + file
  image_np = load_image_into_numpy_array(image_path)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image_np)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis, ...]
  
  detections = detect_fn(input_tensor)
  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(detections.pop('num_detections'))
  detections = {key: value[0, :num_detections].numpy()
                for key, value in detections.items()}
  detections['num_detections'] = num_detections
  
  # detection_classes should be ints.
  detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
  image_np_with_detections = image_np.copy()
  viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_detections,
      detections['detection_boxes'],
      detections['detection_classes'],
      detections['detection_scores'],
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=200,
      min_score_thresh=.4, # Adjust this value to set the minimum probability boxes to be classified as True
      agnostic_mode=False)
  


  try:
    #Create indexes list of element with a score > 0.5
    indexes = [k for k,v in enumerate(detections['detection_scores']) if (v > 0.5)]

    #Number of entities
    num_entities = len(indexes)

    #Extract the class id
    class_id = itemgetter(*indexes)(detections['detection_classes'])
    scores = itemgetter(*indexes)(detections['detection_scores'])


    #Convert the class id in their name
    class_names = []
    if num_entities == 1:
      class_names.append(category_index[class_id]['name'])
      class_names = str(class_names)
    else:
      for i in range(0, len(indexes)):
        class_names.append(category_index[class_id[i]]['name'])
        print(str(scores[i]))
    
  except TypeError:
    class_names = []
    class_names.append('No cat/dog found')
    scores = 0
  print(f'Picture number {file_number}')
  print(f'Detected classes{class_names}')
  print(f'Score : {scores}')
  print(f'file : {file}')


  # add to xlsx
  worksheet.write(f'A{row_index}', file)
  worksheet.write(f'B{row_index}', str(class_names))
  worksheet.write(f'C{row_index}', str(scores))
  worksheet.write(f'D{row_index}', str(file_number))
  row_index += 1
  #fig.add_subplot(rows,columns,file_number)
  file_number+=1
  plt.figure(figsize=IMAGE_SIZE)
  plt.imshow(image_np_with_detections)
  plt.axis("off")
  plt.show()

workbook.close()

In [ ]:
workbook = xlsxwriter.Workbook(f'{report_path}/guesses3_1500_sample.xlsx')
worksheet = workbook.add_worksheet(f'Dog-Cat Guesses')

# Column names
worksheet.write('A1', 'Picture_ID')
worksheet.write('B1', 'Class')
worksheet.write('C1', 'Score')
worksheet.write('D1', 'Picture_Nr')

row_index = 2
file_number = 1

In [ ]:
images_dir_testing = "/mydrive/customTF2/1000_test/"

for file in os.listdir(images_dir_testing):
  image_path = images_dir_testing + file
  image_np = load_image_into_numpy_array(image_path)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image_np)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis, ...]
  
  detections = detect_fn(input_tensor)
  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(detections.pop('num_detections'))
  detections = {key: value[0, :num_detections].numpy()
                for key, value in detections.items()}
  detections['num_detections'] = num_detections
  
  # detection_classes should be ints.
  detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
  image_np_with_detections = image_np.copy()
  viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_detections,
      detections['detection_boxes'],
      detections['detection_classes'],
      detections['detection_scores'],
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=200,
      min_score_thresh=.4, # Adjust this value to set the minimum probability boxes to be classified as True
      agnostic_mode=False)
  


  try:
    #Create indexes list of element with a score > 0.5
    indexes = [k for k,v in enumerate(detections['detection_scores']) if (v > 0.5)]

    #Number of entities
    num_entities = len(indexes)

    #Extract the class id
    class_id = itemgetter(*indexes)(detections['detection_classes'])
    scores = itemgetter(*indexes)(detections['detection_scores'])


    #Convert the class id in their name
    class_names = []
    if num_entities == 1:
      class_names.append(category_index[class_id]['name'])
      class_names = str(class_names)
    else:
      for i in range(0, len(indexes)):
        class_names.append(category_index[class_id[i]]['name'])
        print(str(scores[i]))
    
  except TypeError:
    class_names = []
    class_names.append('No cat/dog found')
    scores = 0
  print(f'Picture number {file_number}')
  print(f'Detected classes{class_names}')
  print(f'Score : {scores}')
  print(f'file : {file}')


  # add to xlsx
  worksheet.write(f'A{row_index}', file)
  worksheet.write(f'B{row_index}', str(class_names))
  worksheet.write(f'C{row_index}', str(scores))
  worksheet.write(f'D{row_index}', str(file_number))
  row_index += 1
  #fig.add_subplot(rows,columns,file_number)
  file_number+=1
  plt.figure(figsize=IMAGE_SIZE)
  plt.imshow(image_np_with_detections)
  plt.axis("off")
  plt.show()

workbook.close()